In [1]:
# !pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic= mp.solutions.holistic      #Detection model
mp_drawing= mp.solutions.drawing_utils  #Drawing connections

In [4]:
def mediapipe_detection(image, model):
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   #Color conversion
    image.flags.writeable= False                    #Image no longer writeable
    results= model.process(image)                   #Make prediction  
    image.flags.writeable= True                     #Image now writeable
    image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)   #Color conversion 
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)      #Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)       #Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)  #Draw left hand Connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS) #Draw right hand Connections

In [6]:
# cap = cv2.VideoCapture(0)

# #Access Mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():
        
#         #read feed from cam(0)
#         ret, frame= cap.read()
        
#         #make detection
#         image, results = mediapipe_detection(frame, holistic)
        
#         #Draw connections on the frame
#         draw_landmarks(image, results)
        
#         #Output connection to the screen
#         cv2.imshow('OpenCV Feed', image)
        
#         #Exit Program with 'q'
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
# cap.release()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [7]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [8]:
#Path for exported data, numpy arrays
DATA_PATH= os.path.join('MP_DATA')

#Actions to detect
words = np.array(['hi','thanks','iloveyou', 'my', 'name'])
alphabets= np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'])
numbers = np.arange(0,9)

actions=np.concatenate([words,alphabets,numbers])

#Thirty videos worth of data
no_sequences= 30

#Video length in frames
sequence_length= 30


In [9]:
#Make folders for data collections
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [10]:
cap = cv2.VideoCapture(0)

#Access Mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through actions
    for action in actions:
        #Loop through videos
        for sequence in range(no_sequences):
            #Loop through frames  
            for frame_num in range(sequence_length):
                 #read feed from cam(0)
                 ret, frame= cap.read()
                
                 #make detection
                 image, results = mediapipe_detection(frame, holistic)
                
                 #Draw connections on the frame
                 draw_landmarks(image, results)
                 
                 #Apply wait logic:
                 if frame_num== 0:
                     cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (132, 63, 195), 4, cv2.LINE_AA)
                     cv2.putText(image, 'Collecting frames for {} Video Number{}'.format(action, sequence), (15, 50),
                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 99, 0), 4, cv2.LINE_AA)
                     cv2.waitKey(1000)
                 else:
                    cv2.putText(image, 'Collecting frames for {} Video Number{}'.format(action, sequence), (15, 50),
                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (130, 99, 0), 4, cv2.LINE_AA)
                     
                 #New Export keypoints
                 keypoints= extract_keypoints(results)
                 npy_path= os.path.join(DATA_PATH, action,str(sequence),str(frame_num))
                 np.save(npy_path, keypoints)
                
                 #Output connection to the screen
                 cv2.imshow('OpenCV Feed', image)
                
                 #Exit Program with 'q'
                 if cv2.waitKey(10) & 0xFF == ord('q'):
                     break 
                 if cv2.waitKey(10) & 0xFF == ord('p'):
                     cv2.putText(image, 'PAUSE', (120, 200),
                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (132, 63, 195), 4, cv2.LINE_AA)
                     cv2.waitKey(120000) 
        
       
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


-1